In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline


import os
from matplotlib import pyplot
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore


import warnings                                                                 
warnings.filterwarnings('ignore') 

# allow plots to appear directly in the notebook
%matplotlib inline

from subprocess import check_output
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.neighbors import KernelDensity

In [ ]:
df = pd.read_csv("avocado.csv")
df.head()

In [ ]:
df.tail()

In [ ]:
df.isnull().sum().sum() #checking for null
#since there are no nulls , we are good to go

In [ ]:
#re-checking for nulls
df.describe()
#since the count for all rows=18249.00, we cna conclude, none of the values are missing in the dataset 

In [ ]:
df.columns 

In [ ]:
#since 'Unnamed: 0' is jut part o the index, we can remove it. 
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

In [ ]:
df.info() #date, type and region is object

In [ ]:
#Above in info, it is observed that Date is Object type not the date type. We have to change its type to date type.
df['Date']=pd.to_datetime(df['Date'])
df['Month']=df['Date'].apply(lambda x:x.month)
df['Day']=df['Date'].apply(lambda x:x.day)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
region_list=list(df.region.unique())
average_price=[]

for i in region_list:
    x=df[df.region==i]
    region_average=sum(x.AveragePrice)/len(x)
    average_price.append(region_average)

df1=pd.DataFrame({'region_list':region_list,'average_price':average_price})
new_index=df1.average_price.sort_values(ascending=False).index.values
sorted_data=df1.reindex(new_index)

plt.figure(figsize=(24,10))
ax=sns.barplot(x=sorted_data.region_list,y=sorted_data.average_price)

plt.xticks(rotation=90)
plt.xlabel('Region')
plt.ylabel('Average Price')
plt.title('Average Price of Avocado According to Region')
#Following places have the hightst proce: HartfordSpringfield, SanFrancisco, NewYork
#Philadelphia and Sacramento

In [ ]:
df_visualization_nominal=df[["region", "type"]].copy() #plotting nominal data

In [ ]:
df_visualization_nominal.columns

In [ ]:
import seaborn as sns
nominal_1=sns.countplot(x="type", data=df_visualization_nominal)
print(df_visualization_nominal["type"].value_counts())

In [ ]:
import seaborn as sns
nominal_1=sns.countplot(x="region", data=df_visualization_nominal)
print(df_visualization_nominal["region"].value_counts())

In [ ]:
#visualising continious columns
df_visualization_continous=df[['Date', 'AveragePrice', 'Total Volume', '4046', '4225', '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'year', 'Month', 'Day']].copy() #plotting nominal data

In [ ]:
df_visualization_continous.columns

In [ ]:
df['region'].nunique() #there are 54 regions. it will be difficult to encode

In [ ]:
#However, since the data is big with 18249, have decided to keep the region column , which will just be 54 types
df.shape

In [ ]:
#plotting average price-target variable as continous columns
import seaborn as sns
sns.distplot(df_visualization_continous["AveragePrice"], kde=True)


In [ ]:
#plotting total volume feature as continous columns
import seaborn as sns
sns.distplot(df_visualization_continous["Total Volume"], kde=True)

In [ ]:
#plotting small bags variable as continous columns
import seaborn as sns
sns.distplot(df_visualization_continous["Total Bags"], kde=True)

In [ ]:
#plotting average price-target variable as continous columns
import seaborn as sns
sns.distplot(df_visualization_continous["4225"], kde=True)

In [ ]:
#plotting average price-target variable as continous columns
import seaborn as sns
sns.distplot(df_visualization_continous["4046"], kde=True)

In [ ]:
#Encoding of DataFrame:
from sklearn.preprocessing import OrdinalEncoder
enc=OrdinalEncoder()

In [ ]:
for i in df.columns:
    if df[i].dtypes == "object":
        df[i]=enc.fit_transform(df[i].values.reshape(-1,1))

In [ ]:
df

In [ ]:
df.isnull().sum().sum()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(22,7))
sns.heatmap(df.describe(),annot=True,linewidths=0.1,linecolor="black",fmt="0.2f")

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(22,7))
sns.heatmap(df.corr(),annot=True,linewidths=0.1,linecolor="black",fmt="0.2f")

In [ ]:
#Checking the columns which are positively and negative correlated with the target column of AVERAGE PRICE - regression
plt.figure(figsize=(22,7))
df.corr()['AveragePrice'].sort_values(ascending=False).drop(["AveragePrice"]).plot(kind="bar",color="c")
plt.xlabel("Feature",fontsize=14)
plt.ylabel('column with target names',fontsize=14)
plt.title('correlation', fontsize=18)
plt.show()

In [ ]:
#Checking the columns which are positively and negative correlated with the target column of REGION - Logistic regression-xlassification
plt.figure(figsize=(22,7))
df.corr()['region'].sort_values(ascending=False).drop(["region"]).plot(kind="bar",color="c")
plt.xlabel("Feature",fontsize=14)
plt.ylabel('column with target names',fontsize=14)
plt.title('correlation', fontsize=18)
plt.show()

In [ ]:
df.skew()

In [ ]:
#Following are the skewed data
#total volume, 4046, 4225, 4770, all bags, - Total Bags, Small Bags, Large Bags, XLarge Bags
#checking for outliers in these fetures

In [ ]:
#Checking the outliers on the int and float type of columns.
df['Total Volume'].plot.box()

In [ ]:
df['4046'].plot.box()

In [ ]:
df['4225'].plot.box()

In [ ]:
df['4770'].plot.box()

In [ ]:
df['Total Bags'].plot.box()

In [ ]:
df['Small Bags'].plot.box()

In [ ]:
df['Large Bags'].plot.box() 

In [ ]:
df['XLarge Bags'].plot.box() 

In [ ]:
#PREDICTING AVERAGE PRICE OF AVACADO

In [ ]:
#Shall be removing the region column as it is categorical data and has 54 types-it will be difficult to calculate the z score
#, which will be difficult and tedious to encode
#Shall also be dropping the date column as day, year and month are already given 

In [ ]:
#Removing Date as month and day are aleady present
df.drop('Date', axis=1, inplace=True)
df.head()

In [ ]:
df

In [ ]:
#Shall be using zscore to remove outliers

In [1]:
!pip install zscore

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for zscore: filename=zscore-0.1-py3-none-any.whl size=1666 sha256=6a0a60caa23995de4646159edc76e3e28a6cb3735abd6df2d60334565d207ada
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\81\fc\f5\963f82253fd63e8eef6b4a1d2b3b2fd90799088ccab9834d61
  Created wheel for statistics: filename=statistics-1.0.3.5-py3-none-any.whl size=7454 sha256=3f749af012d1fb245f8b0f4a12b482f3ce93b6a6e5e06dfc901e0784ed7e7645
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\26\3c\70\9467407f3aa90862061eadcd286627b23a8bab6789b667776f
Successfully built zscore statistics

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from scipy.stats import zscore
import numpy as np
z=np.abs(zscore(df))
threshld=3
np.where(z>3)


In [ ]:
df_new_z=df[(z<3).all(axis=1)]
df_new_z

In [ ]:
df_new_z.shape

In [ ]:
#separating features and taret column
X=df_new_z.drop("AveragePrice", axis=1)
y=df_new_z["AveragePrice"]


In [ ]:
#using lonear regression
from sklearn import metrics
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.model_selection import train_test_split
lr=LinearRegression()

In [ ]:
from sklearn.metrics import r2_score
for i in range(0,100):
    X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.25,random_state=99)
    lr.fit(X_train,y_train)
    pred_train=lr.predict(X_train)
    pred_test=lr.predict(X_test)
    print(f"At random state {i},the training accuracy is:- {r2_score(y_train, pred_train)}")
    print(f"At random state {i},the testing accuracy is:- {r2_score(y_test,pred_test)}")
    print("\n")

In [ ]:
lr=LinearRegression()
lr.fit(X_train,y_train)
pred=lr.predict(X_test)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, pred))
print('MSE:', metrics.mean_squared_error(y_test, pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred)))

In [ ]:
#With descision tree
from sklearn.tree import DecisionTreeRegressor
dt=DecisionTreeRegressor()
dt.fit(X_train,y_train)
pred=dt.predict(X_test)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, pred))
print('MSE:', metrics.mean_squared_error(y_test, pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred)))

In [ ]:
#With random forest
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)
pred=rfr.predict(X_test)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, pred))
print('MSE:', metrics.mean_squared_error(y_test, pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pred)))

In [ ]:
data = pd.DataFrame({'Y Test':y_test , 'Pred':pred},columns=['Y Test','Pred'])
sns.lmplot(x='Y Test',y='Pred',data=data,palette='rainbow')
data.head()

In [ ]:
#FINDING THE REGION

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline


import os
from matplotlib import pyplot
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore


import warnings                                                                 
warnings.filterwarnings('ignore') 

# allow plots to appear directly in the notebook
%matplotlib inline

from subprocess import check_output
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.neighbors import KernelDensity

In [ ]:
df_new_z #this is the data after calculating the zscore as above and removing date as a column.

In [ ]:
X=df_new_z.drop("region", axis=1)
Y=df_new_z["region"]

In [ ]:
#Transforming data to remove skewness:
from sklearn.preprocessing import power_transform

In [ ]:
X=power_transform(X,method="yeo-johnson")

In [ ]:
X

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
Scaled_X=sc.fit_transform(X)
Scaled_X

In [ ]:
from sklearn.model_selection import train_test_split
LR=LogisticRegression()

In [ ]:
#Splitting the dataframe:
for i in range(0,400):
    X_train,X_test,Y_train,Y_test=train_test_split(Scaled_X,Y,random_state=i,test_size=0.25)
    LR.fit(X_train, Y_train)
    pred_train=LR.predict(X_train)
    pred_test=LR.predict(X_test)
    if round(accuracy_score(Y_train, pred_train)*100,1)==round(accuracy_score(Y_test,pred_test)*100,1):
        print("At random_state state",i,"The model performs very well")
        print("At random_state state:-",i)
        print("Training accuracy_score is:-", round(accuracy_score(Y_train,pred_train)*100,1))
        print("Testing accuracy_score is:-", round(accuracy_score(Y_test,pred_test)*100,1), '\n\n')

In [ ]:
X_train,_X_test,Y_train,Y_test=train_test_split(Scaled_X,Y,test_size=0.25, random_state=378)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,pred_test))

In [ ]:
pred_LR=LR.predict(X_test)
from sklearn.model_selection import cross_val_score
lss=accuracy_score(Y_test,pred_LR)


In [ ]:
for j in range(2,10):
    lsscore=cross_val_score(LR,Scaled_X,Y,cv=j)
    lsc=lsscore.mean()
    print("At cv:-",)
    print("Cross validation score is:-", lsc*100)
    print("accuracy_score is:-",lss*100)
    print("\n")

In [ ]:
lsscore_selected=cross_val_score(LR,Scaled_X,Y,cv=7).mean()
print("The cv score is :", lsscore_selected, "\nThe accuracy score is: ",lss)

In [ ]:
#using xgboost to increae accuracy

In [ ]:
!pip install xgboost

In [ ]:
#xgb
import xgboost as xgb
xgb=xgb.XGBRegressor()

In [ ]:
X_train, X_test,Y_train,Y_test=train_test_split(Scaled_X,Y,test_size=0.25, random_state=196)

In [ ]:
xgb.fit(X_train, Y_train) 

In [ ]:
Y_pred=xgb.predict(X_test)

In [ ]:
r2_score(Y_test,Y_pred)

In [ ]:
#USing KNeighbors
from sklearn.neighbors import KNeighborsClassifier  
from time import time
knn = KNeighborsClassifier()
start=time()
knn.fit(X_train, Y_train)
print("knn training time:", (time()-start))
start=time()
Y_pred1=knn.predict(X_test)
print("knn training time:", (time()-start))

In [ ]:
cfm=confusion_matrix(Y_test,Y_pred1)
cfm

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(Y_test,Y_pred1, digits=2))


In [ ]:
#using GridSearchCV
from sklearn.model_selection import GridSearchCV
param_grid = { 'algorithm': ['kd_tree', 'brute'], #it will compare KD_tree leaf size 3 with 3, 5, 7, 9, 11, 13 n_neighbours respectively
'leaf_size': [3,5,6,7,8],
'n_neighbors': [3,5,7,9,11,13]
              }

In [ ]:
gridsearch = GridSearchCV(estimator=knn, param_grid=param_grid)

In [ ]:
gridsearch.fit(X_train,Y_train)

In [ ]:
gridsearch.best_params_

In [ ]:
# we will use the best parameters in our k-NN algorithm and check if accuracy is increasing.
knn=KNeighborsClassifier(algorithm = 'kd_tree', leaf_size=3, n_neighbors=5)

In [ ]:
knn.fit(X_train,Y_train)

In [ ]:
Y_pred1 = knn.predict(X_test)

In [ ]:
cfm = confusion_matrix(Y_test,Y_pred1)
cfm

In [ ]:
print(classification_report(Y_test,Y_pred1, digits=2))

In [ ]:
#GradientBoostingClassifier and #gradient boosting tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
grid = GridSearchCV(GradientBoostingClassifier(), param_grid=grid_param)
grid.fit(X_train,Y_train)

In [ ]:
grid.best_params_

In [ ]:
gbdt_clf=GradientBoostingClassifier(
    max_depth=4, min_samples_split=2, learning_rate=0.1)
gbdt_clf.fit(X_train, Y_train)
pred=gbdt_clf.predict(X_train)
gbdt_c1f_report=pd.DataFrame(
    classification_report(Y_train, pred, output_dict=True))
print("\n================Train Result======================")
print (f"Accuracy Score: {accuracy_score(y_train, pred)*100:.2f}%")
print("____________________________________________________")
print (f"CLASSIFICATION REPORT:\n{gbdt_clf_report}")
print("____________________________________________________")
print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)} \n")

#***********************Test score******************************************
pred=gbdt_clf.predict(X_test)
clf_report = pd.DataFrame(
    classification_report(Y_test, pred, output_dict=True))
print("\n=================Test Result ==========================")
print (f"Accuracy Score: {accuracy_score(Y_test, pred)* 100:.2f}%")
print("____________________________________________________")
print (f"CLASSIFICATION REPORT: \n{clf_report}")
print("____________________________________________________")
print (f"Confusion Matrix: \n {confusion_matrix(Y_test, pred)}\n")


In [ ]:
import numpy as np
a=np.array(Y_test)
predicted=np.array(xgb.predict(X_test))
df_com=pd.DataFrame({"original":a,"predicted":predicted}, index=range(len(a)))
df_com


In [ ]:
import pickle
filename='avocado.pikl'
pickle.dump(lr, open(filename, 'wb'))

In [ ]:
import numpy as np